<a href="https://colab.research.google.com/github/audreychristensen/Bird_Audio_CNN/blob/main/2.1%20Model%20for%20All%20Species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle

import h5py
import numpy as np
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

import librosa
import librosa.display

import IPython.display as ipd
from PIL import Image

import soundfile as sf
import scipy.io.wavfile as wave
import scipy.ndimage as ndimage
import scipy.stats as stats
from scipy import interpolate
import traceback
import tensorflow as tf

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [ ]:
base_dir = '/content/drive/MyDrive/F2024/Applied Data Science/Project 3/'

In [ ]:
hdf5_path = base_dir + 'output_spectrograms_final.h5'
birds_df = pd.read_csv(base_dir + 'bird_dict.csv')

In [ ]:
bird_dict = dict(zip(birds_df.iloc[:, 0], birds_df.iloc[:, 1]))

In [ ]:
# Set runtime to GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
hdf5_paths = [base_dir + 'birds_data_2.h5', base_dir + 'birds_data_3.h5', base_dir + 'birds_data_4.h5', base_dir + 'birds_data_5.h5']

# Define

In [ ]:
class BirdSpectrogramDataset(Dataset):
    def __init__(self, hdf5_paths, bird_dict, transform=None):
        """
        Args:
            hdf5_paths (list): List of paths to HDF5 files.
            bird_dict (dict): Dictionary mapping bird species names to integer labels.
            transform (callable, optional): Optional transform to apply to spectrograms.
        """
        self.hdf5_paths = hdf5_paths
        self.bird_dict = bird_dict
        self.transform = transform
        self.index_map = []  # To store (file_idx, group_name, spectrogram_key)

        # Build an index of all spectrograms across files
        for file_idx, hdf5_path in enumerate(hdf5_paths):
            with h5py.File(hdf5_path, 'r') as f:
                for species_name, species_group in f.items():
                    species_label = bird_dict.get(species_name, -1)  # Get label from dictionary
                    for spectrogram_key in species_group.keys():
                        self.index_map.append((file_idx, species_name, spectrogram_key, species_label))

    def __len__(self):
        return len(self.index_map)

    def __getitem__(self, idx):
        file_idx, species_name, spectrogram_key, label = self.index_map[idx]
        hdf5_path = self.hdf5_paths[file_idx]

        # Load spectrogram lazily from the corresponding HDF5 file
        with h5py.File(hdf5_path, 'r') as f:
            spectrogram = f[species_name][spectrogram_key][()]

        if self.transform:
            spectrogram = self.transform(spectrogram)

        return spectrogram, label


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # You can adjust this if needed
])

In [ ]:
dataset = BirdSpectrogramDataset(hdf5_paths, bird_dict, transform=transform)

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

all_labels = []
for hdf5_path in hdf5_paths:
    with h5py.File(hdf5_path, 'r') as f:
        for species_name, species_group in f.items():
            species_label = bird_dict.get(species_name, -1)
            all_labels.extend([species_label] * len(species_group))

indices = list(range(len(all_labels)))
train_indices, val_indices = train_test_split(
    indices, test_size=0.2, stratify=all_labels
)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Initial Model (2.1)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 32 * 64, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 128 * 32 * 64)  # Flattening
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
num_classes = len(bird_dict)
model = CNNModel(num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
checkpoint_path = base_dir + 'model_checkpoint.pth'

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%')

    torch.save(model.state_dict(), checkpoint_path)
    print(f'Model weights saved to {checkpoint_path}')

Epoch [1/10], Loss: 6.1479, Accuracy: 1.26%
Epoch [2/10], Loss: 4.3136, Accuracy: 1.30%
Epoch [3/10], Loss: 4.3132, Accuracy: 1.33%
Epoch [4/10], Loss: 4.3131, Accuracy: 1.39%
Epoch [5/10], Loss: 4.3130, Accuracy: 1.36%
Epoch [6/10], Loss: 4.3128, Accuracy: 1.37%
Epoch [7/10], Loss: 4.3128, Accuracy: 1.31%
Epoch [8/10], Loss: 4.3126, Accuracy: 1.42%
Epoch [9/10], Loss: 4.3128, Accuracy: 1.32%
Epoch [10/10], Loss: 4.3126, Accuracy: 1.35%


In [ ]:
new_batch_size = 64
new_lr = 0.0005

train_loader = DataLoader(train_dataset, batch_size=new_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=new_batch_size, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=new_lr)

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%')

    torch.save(model.state_dict(), checkpoint_path)
    print(f'Model weights saved to {checkpoint_path}')

Epoch [1/3], Loss: 4.3121, Accuracy: 1.33%
Model weights saved to /content/drive/MyDrive/F2024/Applied Data Science/Project 3/model_checkpoint.pth


KeyboardInterrupt: 

# Model w Updates (2.2)




In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        # Increase fully connected layer sizes
        self.fc1 = nn.Linear(256 * 16 * 32, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))

        x = x.view(-1, 256 * 16 * 32)  # Adjust flattening
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
num_classes = len(bird_dict)
model2 = CNNModel(num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.0005)

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%')

    torch.save(model2.state_dict(), checkpoint_path)
    print(f'Model weights saved to {checkpoint_path}')

Epoch [1/5], Loss: 4.3687, Accuracy: 1.56%
Model weights saved to /content/drive/MyDrive/F2024/Applied Data Science/Project 3/model_checkpoint.pth
Epoch [2/5], Loss: 4.3097, Accuracy: 1.55%
Model weights saved to /content/drive/MyDrive/F2024/Applied Data Science/Project 3/model_checkpoint.pth
Epoch [3/5], Loss: 4.3130, Accuracy: 1.32%
Model weights saved to /content/drive/MyDrive/F2024/Applied Data Science/Project 3/model_checkpoint.pth
Epoch [4/5], Loss: 4.3128, Accuracy: 1.30%
Model weights saved to /content/drive/MyDrive/F2024/Applied Data Science/Project 3/model_checkpoint.pth


KeyboardInterrupt: 

# Further updates (2.3)

In [ ]:
class BirdClassifierCNN(nn.Module):
    def __init__(self, num_classes):
        super(BirdClassifierCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.4)

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))

        x = self.gap(x).view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)

        return x

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)


In [ ]:
learning_rate = 0.001
batch_size = 32

In [ ]:
model_checkpoint_path = base_dir + 'model_checkpoint_3.2.pth'
optimizer_checkpoint_path = base_dir + 'optimizer_checkpoint_3.2.pth'
scheduler_checkpoint_path = base_dir + 'scheduler_checkpoint_3.2.pth'

In [ ]:
num_classes = len(bird_dict)
model = BirdClassifierCNN(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)  # Reinitialize scheduler

In [ ]:
for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%')

        # Save model weights after each epoch
        torch.save(model.state_dict(), model_checkpoint_path)
        torch.save(optimizer.state_dict(), optimizer_checkpoint_path)

Epoch [1/5], Loss: 4.1823, Accuracy: 3.60%
Epoch [2/5], Loss: 3.9657, Accuracy: 6.67%
Epoch [3/5], Loss: 3.7798, Accuracy: 9.79%
Epoch [4/5], Loss: 3.6223, Accuracy: 12.79%
Epoch [5/5], Loss: 3.4839, Accuracy: 15.93%


In [ ]:
start_epoch = 10
num_epochs = 20

In [ ]:
for epoch in range(start_epoch, num_epochs):  # start_epoch is the epoch to resume from
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Step the scheduler after each epoch to adjust the learning rate
    scheduler.step()

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%')

    # Save model weights, optimizer state, and scheduler state after each epoch
    torch.save(model.state_dict(), model_checkpoint_path)
    torch.save(optimizer.state_dict(), optimizer_checkpoint_path)
    torch.save(scheduler.state_dict(), scheduler_checkpoint_path)  # Save scheduler state


Epoch [11/20], Loss: 3.0158, Accuracy: 24.91%
Epoch [12/20], Loss: 2.9576, Accuracy: 26.16%


# Quick Validate

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = 100 * correct / total
print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Accuracy: 24.27%


# Validation Accuracy per Species:

In [ ]:
model.eval()
correct = 0
total = 0
species_correct = {species: 0 for species in bird_dict.keys()}
species_total = {species: 0 for species in bird_dict.keys()}

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Track correct and total for each species
        for i in range(labels.size(0)):
            species_name = list(bird_dict.keys())[list(bird_dict.values()).index(labels[i].item())]
            species_total[species_name] += 1
            if predicted[i] == labels[i]:
                species_correct[species_name] += 1

val_accuracy = 100 * correct / total
print(f'Validation Accuracy: {val_accuracy:.2f}%')

# Calculate and print % of correct classifications per species
print("\nPercentage of Correct Classifications per Species:")
for species in bird_dict.keys():
    species_accuracy = 100 * species_correct[species] / species_total[species] if species_total[species] > 0 else 0
    print(f"Species: {species}, Correct: {species_accuracy:.2f}%")


Validation Accuracy: 24.27%

Percentage of Correct Classifications per Species:
Species: NorthernGoshawk, Correct: 3.96%
Species: WaterPipit, Correct: 47.52%
Species: PeregrineFalcon, Correct: 38.10%
Species: HoodedWarbler, Correct: 85.85%
Species: Ovenbird, Correct: 19.81%
Species: HouseSparrow, Correct: 0.00%
Species: WoodThrush, Correct: 23.30%
Species: HouseWren, Correct: 12.62%
Species: TundraSwan, Correct: 21.57%
Species: HornedLark, Correct: 65.35%
Species: Chestnut-sidedWarbler, Correct: 2.88%
Species: Gull-billedTern, Correct: 9.28%
Species: GreatEgret, Correct: 28.43%
Species: Gadwall, Correct: 19.80%
Species: CommonGoldeneye, Correct: 16.83%
Species: GreatHornedOwl, Correct: 60.00%
Species: LesserBlack-backedGull, Correct: 38.83%
Species: NorthernCardinal, Correct: 17.92%
Species: RedCrossbill, Correct: 65.14%
Species: RoseateTern, Correct: 33.02%
Species: GreatCormorant, Correct: 4.81%
Species: Red-eyedVireo, Correct: 1.92%
Species: WarblingVireo, Correct: 0.00%
Species: Gr

In [ ]:
class_to_species = {v: k for k, v in bird_dict.items()}

In [ ]:
with open('class_to_species.pkl', 'wb') as f:
    pickle.dump(class_to_species, f)

In [ ]:
torch.save(model.state_dict(), base_dir + 'full_bird_model.pth')

# Now we try to predict a bird from a full length recording! I will be using a recording >1 minute long because it is easy for me to know that that didn't make it into our training data

In [ ]:
def getMelSpec(path, seconds = 5, overlap = 4, minlen = 3, winlen=0.05, winstep=0.0097, NFFT=840, sr_target=44100):
  """
  """
  y, sr = librosa.load(path, sr=sr_target)
  print(f"shape: {y.shape}", f'sr: {sr}')
  sig_splits = []
  step = int((seconds - overlap) * sr)
  window_length = int(seconds * sr)
  NFFT = max(NFFT, int(winlen * sr))

  for i in range(0, len(y), step):
    split =  y[i:i + window_length]
    if len(split) >= minlen:
      sig_splits.append(split)

  if len(sig_splits) == 0:
    sig_splits.append(sig)


  for split_sig in sig_splits:
        # compute mel spectrogram
        mel_spec = librosa.feature.melspectrogram(
            y=split_sig,
            sr=sr,
            n_fft=int(winlen * sr),
            hop_length=int(winstep * sr),
            n_mels=128*2
        )

        # Convert to dB scale
        mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)

        # resize to fixed shape
        mel_spec_resized = cv2.resize(mel_spec_db, (512, 256))

        yield mel_spec_resized

def filter_isolated_cells(array, struct):

    filtered_array = np.copy(array)
    id_regions, num_ids = ndimage.label(filtered_array, structure=struct)
    id_sizes = np.array(ndimage.sum(array, id_regions, range(num_ids + 1)))
    area_mask = (id_sizes == 1)
    filtered_array[area_mask[id_regions]] = 0

    return filtered_array


def hasBird(spec, threshold=16):

    #working copy
    img = spec.copy()

    #STEP 1: Median blur
    img = cv2.medianBlur(img,5)

    #STEP 2: Median threshold
    col_median = np.median(img, axis=0, keepdims=True)
    row_median = np.median(img, axis=1, keepdims=True)

    img[img < row_median * 3] = 0
    img[img < col_median * 4] = 0
    img[img > 0] = 1

    #STEP 3: Remove singles
    img = filter_isolated_cells(img, struct=np.ones((3,3)))

    #STEP 4: Morph Closing
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, np.ones((5,5), np.float32))

    #STEP 5: Frequency crop
    img = img[128:-16, :]

    #STEP 6: Count columns and rows with signal
    #(Note: We only use rows with signal as threshold, but columns might come in handy in other scenarios)

    #column has signal?
    col_max = np.max(img, axis=0)
    col_max = ndimage.morphology.binary_dilation(col_max, iterations=2).astype(col_max.dtype)
    cthresh = col_max.sum()

    #row has signal?
    row_max = np.max(img, axis=1)
    row_max = ndimage.morphology.binary_dilation(row_max, iterations=2).astype(row_max.dtype)
    rthresh = row_max.sum()

    #final threshold
    thresh = rthresh

    #DBUGB: show?
    #print thresh
    #cv2.imshow('BIRD?', img)
    #cv2.waitKey(-1)

    #STEP 7: Apply threshold (Default = 16)
    bird = True
    if thresh < threshold:
        bird = False

    return bird, thresh

In [ ]:
bird = '/content/drive/MyDrive/F2024/Applied Data Science/Project 3/bird_calls_highest_quality/AmericanRobin/543354.mp3'

In [ ]:
spectrograms = list(getMelSpec(bird))

shape: (2840832,) sr: 44100


In [ ]:
import torch
import numpy as np

def spectrograms_to_tensor(spectrograms, device='cuda'):
    spectrograms_tensor = torch.tensor(np.array(spectrograms)).unsqueeze(1).float().to(device)
    return spectrograms_tensor

def predict_from_spectrograms(model, spectrograms, device='cuda'):
    spectrograms_tensor = spectrograms_to_tensor(spectrograms, device)

    model.eval()

    with torch.no_grad():
        outputs = model(spectrograms_tensor)
        _, predictions = torch.max(outputs, 1)

    return predictions


spectrograms = list(getMelSpec(bird))
predictions = predict_from_spectrograms(model, spectrograms, device='cuda')


print(f'Predictions: {predictions}')


shape: (2840832,) sr: 44100
Predictions: tensor([48, 48,  4,  4,  4,  4,  4,  4,  4, 48, 48, 48, 48,  4, 48,  4,  4,  4,
        10, 10, 10, 10, 10, 10, 48, 48, 48, 48, 10, 10, 10, 25, 10, 25, 10, 10,
         4,  4,  4,  4, 48, 48,  4, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48,
         4,  4,  4,  4,  4,  4,  4, 10, 10, 10, 10], device='cuda:0')


In [ ]:
from collections import Counter

In [ ]:
def get_most_frequent_species(predictions, class_to_species):
    prediction_counts = Counter(predictions.cpu().numpy())

    most_frequent_index = prediction_counts.most_common(1)[0][0]
    most_frequent_count = prediction_counts.most_common(1)[0][1]

    species = class_to_species[most_frequent_index]

    confidence = (most_frequent_count / len(predictions)) * 100

    return species, confidence, prediction_counts

In [ ]:
species, confidence, prediction_counts = get_most_frequent_species(predictions, class_to_species)

print(f'Most frequent species: {species} in {confidence:.2f}% of recording')
print(f'Prediction counts: {dict(prediction_counts)}')



Most frequent species: Dark-eyedJunco in 36.92% of recording
Prediction counts: {48: 24, 4: 23, 10: 16, 25: 2}
